# How to make model

In [1]:
import sys

import torch
import torch.nn as nn
print(sys.version)
print(torch.__version__)

3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
1.7.1


# 1. Wrap all step in `main`

In [2]:
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3)),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3)),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(3,3)),
            nn.ReLU(),
        )

    def forward(self, input):
        return self.main(input)
        
m1 = Model1()
print(m1)

Model1(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 10, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
)


# 2. Attribute every single step

In [3]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3))
        self.acti1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3))
        self.acti2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(3,3))
        self.acti3 = nn.ReLU()
    
    def forward(self, input):
        output = input # or Deep copy ?
        output = self.conv1(output)
        output = self.acti1(output)
        output = self.conv2(output)
        output = self.acti2(output)
        output = self.conv3(output)
        output = self.acti3(output)
        return output

m2 = Model2()
print(m2)

Model2(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (acti1): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (acti2): ReLU()
  (conv3): Conv2d(64, 10, kernel_size=(3, 3), stride=(1, 1))
  (acti3): ReLU()
)


# 3. Wrap up each layers

In [4]:
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3)),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3)),
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(3,3)),
            nn.ReLU(),
        )

    def forward(self, input):
        output = input
        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)
        return output
        
m3 = Model3()
print(m3)


Model3(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (layer3): Sequential(
    (0): Conv2d(64, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
)


## 장점 : 각 layer 별로 parameters를 끌어오기 편함.
 
<=> Update할 parameters 정하기 쉽다.

In [5]:
print(m1.parameters())
print(m2.parameters())
print(m3.parameters())
print()

# One step (conv1)
print(m1.main[0].parameters())
print(m2.conv1.parameters())
print(m3.layer1[0].parameters())


<generator object Module.parameters at 0x000001D0CA34BEB8>
<generator object Module.parameters at 0x000001D0CA34BEB8>
<generator object Module.parameters at 0x000001D0CA34BEB8>

<generator object Module.parameters at 0x000001D0CA34BEB8>
<generator object Module.parameters at 0x000001D0CA34BEB8>
<generator object Module.parameters at 0x000001D0CA34BEB8>


# 3+. Advanced layer model

## Layers로 묶는 방법

1. `nn.Sequential`로 묶는다.
2. `nn.ModuleList` / `nn.ModuleDict`로 묶는다.  
    Python에 원래 있는 `List`, `Dict`를 사용하면 `parameters()` 사용 불가능.  
    꼭 Module로 감싸줘야 한다.

In [6]:
class Model3_1(nn.Module):
    def __init__(self):
        super(Model3_1, self).__init__()
        layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3)),
            nn.ReLU(),
        )
        layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3)),
            nn.ReLU(),
        )
        layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(3,3)),
            nn.ReLU(),
        )
        
        self.layers = nn.ModuleList([layer1, layer2, layer3]) # Forwarding X
        # self.layers = nn.Sequential(layer1, layer2, layer3) # Forwarding O

    def forward(self, input):
        output = input
        # ModuleList
        for layer in layers:
            output = layer(output)
        # # Sequential
        # output = self.layers(output)
        return output
        
m3_1 = Model3_1()
print(m3_1)


Model3_1(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(64, 10, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
    )
  )
)


In [7]:
m3_1.layers[0][0].parameters()

<generator object Module.parameters at 0x000001D0CA34BE60>